In [15]:
import os
import tifffile
import numpy as np 
import pandas as pd
import lightgbm as lgb 

In [16]:
images = os.listdir(r'C:\Users\Naoki Mnagawa\Desktop\Solafne\太陽光パネル\data\train\s2_image')
masks  = os.listdir(r'C:\Users\Naoki Mnagawa\Desktop\Solafne\太陽光パネル\data\train\mask')
print(images)

['train_s2_image_0.tif', 'train_s2_image_1.tif', 'train_s2_image_10.tif', 'train_s2_image_100.tif', 'train_s2_image_1000.tif', 'train_s2_image_1001.tif', 'train_s2_image_1002.tif', 'train_s2_image_1003.tif', 'train_s2_image_1004.tif', 'train_s2_image_1005.tif', 'train_s2_image_1006.tif', 'train_s2_image_1007.tif', 'train_s2_image_1008.tif', 'train_s2_image_1009.tif', 'train_s2_image_101.tif', 'train_s2_image_1010.tif', 'train_s2_image_1011.tif', 'train_s2_image_1012.tif', 'train_s2_image_1013.tif', 'train_s2_image_1014.tif', 'train_s2_image_1015.tif', 'train_s2_image_1016.tif', 'train_s2_image_1017.tif', 'train_s2_image_1018.tif', 'train_s2_image_1019.tif', 'train_s2_image_102.tif', 'train_s2_image_1020.tif', 'train_s2_image_1021.tif', 'train_s2_image_1022.tif', 'train_s2_image_1023.tif', 'train_s2_image_1024.tif', 'train_s2_image_1025.tif', 'train_s2_image_1026.tif', 'train_s2_image_1027.tif', 'train_s2_image_1028.tif', 'train_s2_image_1029.tif', 'train_s2_image_103.tif', 'train_s2_im

In [17]:
maskdir=r'C:\Users\Naoki Mnagawa\Desktop\Solafne\太陽光パネル\data\train\mask'
imgdir=r'C:\Users\Naoki Mnagawa\Desktop\Solafne\太陽光パネル\data\train\s2_image'
img = tifffile.imread(os.path.join(maskdir, masks[1]))
img.shape 

(23, 23)

In [18]:
data_list = []

for i in range(len(images)):
    img = tifffile.imread(os.path.join(imgdir, images[i])).astype(float)
    msk = tifffile.imread(os.path.join(maskdir, masks[i])).astype(float)

    reshaped_img = img.reshape(-1, 12) # (23, 23, 12) -> (23*23, 12)
    flatten_mask = msk.reshape(-1,1)   # (23, 23) -> (23*23, 1)

    is_train= 1
    if i % 5 == 0:  
        is_train= 0
    is_train_image = np.full((reshaped_img.shape[0], 1), is_train)
    
    combine_data = np.hstack((reshaped_img, flatten_mask))
    combine_data = np.hstack((combine_data, is_train_image))

    data_list.append(combine_data)

data_list = np.vstack(data_list)
columns   = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'mask', 'is_train'] 
df        = pd.DataFrame(data_list, columns=columns)
df

,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,mask,is_train
0,265.800583,279.066472,488.098252,331.572300,817.002914,2400.280728,3084.914212,3236.613360,3332.798479,3251.691757,1598.907575,818.413402,0.0,0.0
1,265.800583,271.067287,472.501621,322.083156,817.002914,2400.280728,3084.914212,3188.111508,3332.798479,3251.691757,1598.907575,818.413402,0.0,0.0
2,265.800583,240.942322,424.775329,279.931718,784.908433,2322.216009,2998.675982,2906.248048,3251.978778,3251.691757,1549.296809,798.364396,0.0,0.0
3,265.800583,222.820502,411.740847,245.060190,737.346930,2206.530216,2870.877686,2640.218320,3132.210318,3251.691757,1475.777544,768.653340,0.0,0.0
4,378.735906,229.331874,434.090056,254.714547,782.884060,2226.587592,2840.185718,2651.574272,3084.218565,3341.757199,1562.631939,809.818550,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133567,1166.000000,1049.347260,1190.897532,1254.509945,1365.000000,1379.000000,1468.000000,1463.018392,1461.000000,1490.000000,2027.000000,1859.000000,0.0,0.0
1133568,1166.000000,1090.473893,1243.402564,1362.630769,1365.000000,1379.000000,1468.000000,1572.583217,1461.000000,1490.000000,2027.000000,1859.000000,0.0,0.0
1133569,1089.344375,1097.961592,1251.727616,1360.179232,1399.362866,1489.137392,1583.130287,1621.304781,1591.402672,1639.199454,2009.671717,1797.029361,0.0,0.0
1133570,905.000000,1018.380712,1222.829764,1265.576354,1482.000000,1754.000000,1860.000000,1952.131222,1905.000000,1998.000000,1968.000000,1648.000000,0.0,0.0


In [19]:
df['mask'].value_counts()

0.0    1027487
1.0     106085
Name: mask, dtype: int64

In [20]:
f = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12']
t = 'mask'

X_train = df[df['is_train']==1][f].values
X_test  = df[df['is_train']==0][f].values 

y_train = df[df['is_train']==1][t].values
y_test  = df[df['is_train']==0][t].values 

In [21]:
X_train

array([[ 551.        ,  637.40797886,  845.1312968 , ..., 2528.        ,
        2347.        , 1550.        ],
       [ 551.        ,  663.86593212,  820.72841927, ..., 2528.        ,
        2265.55711564, 1512.06076816],
       [ 551.        ,  667.79485771,  793.89998037, ..., 2528.        ,
        2025.        , 1400.        ],
       ...,
       [ 555.        ,  323.96550346,  542.32736395, ..., 3258.        ,
        1401.31660152,  605.20463325],
       [ 537.32414749,  336.14234378,  514.19158467, ..., 3246.21609832,
        1416.70007278,  615.78634748],
       [ 535.5       ,  335.37558981,  505.62036419, ..., 3245.        ,
        1418.28764659,  616.87838009]])

In [22]:
params = {
    'n_estimators': 1000,
    'learning_rate': 0.01 
}
model = lgb.LGBMClassifier(**params)
model.fit(X_train, y_train, 
          eval_set = [(X_test, y_test)]
          )

[LightGBM] [Info] Number of positive: 83196, number of negative: 822667
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.028768 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3060
[LightGBM] [Info] Number of data points in the train set: 905863, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.091842 -> initscore=-2.291352
[LightGBM] [Info] Start training from score -2.291352


LGBMClassifier(learning_rate=0.01, n_estimators=1000)

In [74]:
from sklearn.svm import SVC
model = SVC(kernel='linear')
model.fit(X_train, y_train)

In [11]:
ev_imgdir=r'C:\Users\Naoki Mnagawa\Desktop\Solafne\太陽光パネル\data\evaluation'
ev_mskdir=r'C:\Users\Naoki Mnagawa\Desktop\Solafne\太陽光パネル\data\sample'
images = os.listdir(r'C:\Users\Naoki Mnagawa\Desktop\Solafne\太陽光パネル\data\evaluation')
masks  = os.listdir(r'C:\Users\Naoki Mnagawa\Desktop\Solafne\太陽光パネル\data\sample')

images.sort()
masks.sort()

In [9]:
if not os.path.isdir(r'C:\Users\Naoki Mnagawa\Desktop\Solafne\太陽光パネル\data\output'):
    os.mkdir(r'C:\Users\Naoki Mnagawa\Desktop\Solafne\太陽光パネル\data\output')

outputpath=r'C:\Users\Naoki Mnagawa\Desktop\Solafne\太陽光パネル\data\output'

In [12]:
data_list = []

for i in range(len(images)):
    img = tifffile.imread(os.path.join(ev_imgdir, images[i])).astype(float)
    msk = tifffile.imread(os.path.join(ev_mskdir, masks[i])).astype(float)

    reshaped_img = img.reshape(-1, 12) # (23, 23, 12) -> (23*23, 12)
    name_image   = np.full((reshaped_img.shape[0], 1), masks[i])
    shape_image  = np.full((reshaped_img.shape[0], 2), (msk.shape[0], msk.shape[1]))

    combine_data = np.hstack((reshaped_img, name_image))
    combine_data = np.hstack((combine_data, shape_image))


    data_list.append(combine_data)

data_list = np.vstack(data_list)
columns   = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'name', 'w', 'h'] 
df        = pd.DataFrame(data_list, columns=columns)
df

,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,name,w,h
0,406.0,313.8514897250669,558.2467410859184,366.5428577132614,875.0,2337.9999999999995,2957.0,3300.8659893147706,3296.0,2964.5000000000005,1701.5,878.0,evaluation_mask_0.tif,24,23
1,406.0,301.6436896965586,521.7770858080386,342.0241547345142,861.0780185343897,2334.6791603843494,2959.2990428108346,3350.1981931152923,3285.654307351244,2964.5,1683.490831315128,859.8631067145261,evaluation_mask_0.tif,24,23
2,401.15295690885637,276.55352604468305,455.96931323949576,301.9294551797059,818.6312604949809,2315.6368364383934,2950.8554235947404,3259.5292064420373,3240.4332877407824,2944.5862446496394,1624.0311589291994,802.6590738581325,evaluation_mask_0.tif,24,23
3,361.28633398411125,249.07326641312625,391.8893698781878,263.8425161370124,772.2193708903116,2214.141803426966,2808.943713905994,2717.1642197419383,3103.615113531097,2927.4067855638973,1497.6434817995118,735.4289909828183,evaluation_mask_0.tif,24,23
4,320.1711241452216,234.58589503877465,366.34324718893475,244.39973362706888,714.4986681353445,2101.694872321076,2657.079568681661,2459.259349947143,2958.4178213781383,2956.2356555603988,1345.368993161773,656.0842302079554,evaluation_mask_0.tif,24,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129963,765.9142987943302,913.4417840527478,1004.5596657125997,1073.3924669236621,1323.4945012527305,1475.446380803405,1560.4656289831353,1579.4404981989544,1696.7030266188563,1913.0751615126887,2024.8888651649934,1631.975481973779,evaluation_mask_999.tif,23,23
1129964,768.5677765853243,895.4359852781945,1075.8476074491014,1174.0312255003585,1415.385725460275,1726.4690098670476,1840.3253774901755,1878.6932394764815,1997.3538522711874,1921.5180453931239,2258.1774481256202,1761.949101345473,evaluation_mask_999.tif,23,23
1129965,769.2667277068745,889.2566458154513,1086.4945012527307,1192.2373976357212,1439.5907421513814,1792.5907421513816,1914.043079503614,1945.7380843380581,2076.548120449326,1923.7419807798747,2319.627865106168,1796.185380744579,evaluation_mask_999.tif,23,23
1129966,769.2667277068743,902.6170774167623,1102.9506249704266,1201.3901366278944,1439.327844075443,1795.966580586991,1918.733170268842,2250.6017046144575,2082.182632398496,1923.7419807798744,2323.679573574237,1797.3201203247095,evaluation_mask_999.tif,23,23


In [13]:
tifffile.imread(r'C:\Users\Naoki Mnagawa\Desktop\Solafne\太陽光パネル\data\sample\evaluation_mask_13.tif').astype(float).shape
df[df['name'] == 'evaluation_mask_13.tif']

,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,name,w,h
183095,294.5,280.30856096933985,587.43513169093,389.1125008341014,1042.2874004858936,2545.465125521721,3165.808772192119,3419.832366805189,3577.9263165763564,3457.5,1837.8211574160282,904.5871861956971,evaluation_mask_13.tif,23,23
183096,294.5,287.61534353865363,564.0126527828359,393.2434733595188,1033.5,2514.5,3151.9999999999995,3409.2322890698133,3536.5,3457.4999999999995,1919.0,981.9999999999999,evaluation_mask_13.tif,23,23
183097,294.5,285.55767796758784,560.9261326518595,386.288451760514,1033.5,2514.5000000000005,3152.0000000000005,3409.783907012393,3536.5,3457.5000000000005,1919.0000000000002,982.0000000000001,evaluation_mask_13.tif,23,23
183098,294.5,296.5672139610336,567.2594901939857,393.61776101442894,1063.8782533189726,2438.2485453604313,2963.0024105591433,3319.542495908975,3350.3567430857584,3457.5,1997.9019062714256,1081.6977575367623,evaluation_mask_13.tif,23,23
183099,294.5,348.89877836872256,594.3936404740062,468.5389174617862,1107.9999999999998,2327.5,2688.5,2950.9832792807333,3079.9999999999995,3457.4999999999995,2112.5,1226.4999999999998,evaluation_mask_13.tif,23,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183619,407.53142948887603,388.541871959504,609.1105977271052,476.33506305849517,976.171010462429,2231.8745592547493,2771.2292151943384,3196.336475377653,3088.1309460646785,3064.120786499408,1693.3818275551791,1036.2849250339636,evaluation_mask_13.tif,23,23
183620,390.0,393.80971571096484,641.1082144029062,494.7382451550636,910.5,2375.0,3000.0,3623.69780516741,3358.0,3250.4999999999995,1677.0,902.5,evaluation_mask_13.tif,23,23
183621,390.0,408.1141717253256,658.7133671899534,515.7108711465037,910.4999999999999,2374.9999999999995,3000.0,3677.426088106548,3357.9999999999995,3250.4999999999995,1677.0,902.5,evaluation_mask_13.tif,23,23
183622,386.1709736383635,352.1861999031695,558.9942924843448,432.23049709300966,943.525352369115,2547.306186273643,3131.1441528860505,3384.9373704477393,3539.878752177734,3247.1496019335677,1785.1699947162315,900.1068585239772,evaluation_mask_13.tif,23,23


In [14]:
X = df[f].values
preds = model.predict_proba(X)

NameError: name 'f' is not defined

In [ ]:
preds = np.argmax(preds, axis=1)
df['mask'] = preds
df['h'] = df['h'].astype('int')
df['w'] = df['w'].astype('int')
for i in range(len(masks)):
    df_tmp     = df[df['name'] == masks[i]].reset_index(drop=True)
    preds_mask = df_tmp['mask'].values.astype(np.uint8)
    preds_mask = preds_mask.reshape(df_tmp['h'][0], df_tmp['w'][0])
    tifffile.imwrite(os.path.join(outputpath, df_tmp['name'][0]), preds_mask)

==============================================================================================================================

　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　test作成領域

==============================================================================================================================

In [57]:
X_train=[]
X_test=[]
y_train=[]
y_test=[]
for i in range(len(images)):
    img = tifffile.imread(os.path.join(imgdir, images[i])).astype(float).tolist()
    msk = tifffile.imread(os.path.join(maskdir, masks[i])).astype(float).tolist()
    
    if i%5==0:
        X_test.append(img)
        y_test.append(msk)
    else:
        X_train.append(img)
        y_train.append(msk)

X_train=np.array(X_train,dtype='float32')
X_test=np.array(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (1652,) + inhomogeneous part.

In [52]:
X_train

array([list([[[551.0, 637.4079788629132, 845.131296798611, 922.7047507095666, 1166.0, 1716.0, 1967.9999999999998, 2246.2565379530624, 2152.0, 2527.9999999999995, 2346.9999999999995, 1550.0], [551.0, 663.8659321157204, 820.7284192668483, 810.7592849452286, 1186.7401134075074, 1738.7635391058009, 1972.8056360334467, 1727.91063488129, 2153.5175692737203, 2527.9999999999995, 2265.55711564369, 1512.0607681569984], [551.0, 667.7948577055363, 793.8999803746436, 802.9361893718293, 1248.0, 1805.9999999999998, 1987.0000000000002, 1775.30650645552, 2157.9999999999995, 2528.0, 2025.0, 1400.0000000000002], [551.0, 657.1751509370541, 813.9968777825677, 847.4856293776973, 1269.0773510292984, 2091.3873329366966, 2333.090103901075, 2280.974504520929, 2531.9122072597484, 2528.0, 2100.8784637054737, 1435.831496749807], [551.0, 537.6297269864267, 768.3164257690403, 736.6805381974812, 1298.0, 2483.0, 2808.0, 3120.767499600832, 3045.0000000000005, 2528.0, 2205.0000000000005, 1485.0000000000002], [551.000000

In [56]:
for i in range(len(X_train)):
    X_train[i]=np.array(X_train[i])
    y_train[i]=np.array(y_train[i])
for i in range(len(X_test)):
    X_test[i]=np.array(X_test[i])
    y_test[i]=np.array(y_test[i])

ValueError: setting an array element with a sequence.

In [58]:

# Kerasの必要なコンポーネントをインポート
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils


model = Sequential()

In [65]:
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu'
                 #input_shape=(,,12)
                 ))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

model.fit(
    X_train,
    y_train,
    epochs = 15,
    batch_size = 30
)

ValueError: Input 0 of layer "conv2d_3" is incompatible with the layer: expected min_ndim=4, found ndim=2. Full shape received: (None, 10)